In [ ]:
!pip install contextily


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 36.0 MB/s eta 0:00:00


In [ ]:
!pip install pysal

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.6/141.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import pysal as ps
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap
import rasterio
from rasterio.features import rasterize
from esda.moran import Moran
from libpysal.weights import Queen
from libpysal.weights import KNN
from splot.esda import plot_moran
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import folium
from folium.plugins import HeatMap
import contextily as ctx

In [ ]:

# 1. Data Loading and Initial Exploration
print("Loading datasets...")

# Load COVID-19 vaccination data by ZIP code
covid_df = pd.read_csv('/content/covid19vaccinesbyzipcode_test.csv')
print(f"COVID-19 data shape: {covid_df.shape}")
print("COVID-19 data first few rows:")
print(covid_df.head())

# Load Income limits by county
income_df = pd.read_csv('/content/2023-income-limits.csv')
print(f"Income data shape: {income_df.shape}")
print("Income data first few rows:")
print(income_df.head())

# Load California counties shapefile
counties_gdf = gpd.read_file('/content/California_Counties.csv')
print(f"Counties shapefile shape: {counties_gdf.shape}")
print("Counties data first few rows:")
print(counties_gdf.head())


Loading datasets...


FileNotFoundError: [Errno 2] No such file or directory: '/content/covid19vaccinesbyzipcode_test.csv'

In [ ]:
# 2. Data Preprocessing

# Examining data types and cleaning
print("\nExamining data types and basic statistics:")
print(covid_df.info())
print(covid_df.describe())

# Check for missing values
print("\nMissing values in COVID data:")
print(covid_df.isnull().sum())
print("\nMissing values in income data:")
print(income_df.isnull().sum())

# Filter to most recent data for each ZIP code (if time series)
covid_df['as_of_date'] = pd.to_datetime(covid_df['as_of_date'])
latest_covid_data = covid_df.sort_values('as_of_date').groupby('zip_code_tabulation_area').last().reset_index()
print(f"\nFiltered to latest data per ZIP code: {latest_covid_data.shape}")



Examining data types and basic statistics:


NameError: name 'covid_df' is not defined

In [ ]:
# 3. Spatial Data Processing
print("\nProcessing spatial data...")

# Try to load a proper California counties shapefile with geometry
try:
    # Option 1: Try loading from a standard source
    counties_gdf = gpd.read_file('https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/california-counties.geojson')
    print("Loaded California counties shapefile from online source")
except:
    print("Could not load counties shapefile from online source, trying alternative approach")
    # Option 2: If counties_df already has the data but needs conversion
    counties_df = pd.read_csv('/content/California_Counties.csv')

    # Check if there might be geometry data in the CSV
    if 'Shape_Area' in counties_df.columns and 'Shape_Length' in counties_df.columns:
        print("Found shape measurements but no actual geometry data")

        # If you have OBJECTID and NAME, you can still work with the data
        if 'OBJECTID' in counties_df.columns and 'NAME' in counties_df.columns:
            # Create a simple point geometry at California's center as placeholder
            # This is just to have a geometric structure - not accurate boundaries
            geometry = [Point(-119.4179, 36.7783) for _ in range(len(counties_df))]
            counties_gdf = gpd.GeoDataFrame(counties_df, geometry=geometry, crs="EPSG:4326")
            print("Created GeoDataFrame with placeholder geometry")
        else:
            print("Warning: Cannot create proper GeoDataFrame due to missing data")
            counties_gdf = gpd.GeoDataFrame(counties_df)
    else:
        print("No geometry information found in counties data")
        counties_gdf = gpd.GeoDataFrame(counties_df)

# Print information about the counties GeoDataFrame
print(f"Counties GeoDataFrame info:")
print(f"- Number of counties: {len(counties_gdf)}")
print(f"- Columns: {counties_gdf.columns.tolist()}")
print(f"- Has geometry column: {'geometry' in counties_gdf.columns}")

# Create a mapping from ZIP codes to counties
# Use the county information already in the COVID data
zip_to_county = latest_covid_data[['zip_code_tabulation_area', 'county']].drop_duplicates()
print("ZIP to county mapping (sample):")
print(zip_to_county.head())


Processing spatial data...
Loaded California counties shapefile from online source
Counties GeoDataFrame info:
- Number of counties: 58
- Columns: ['name', 'cartodb_id', 'created_at', 'updated_at', 'geometry']
- Has geometry column: True


NameError: name 'latest_covid_data' is not defined

In [ ]:
# 4. Aggregate vaccination data to county level
print("\nAggregating vaccination data to county level...")

# Group by county and calculate statistics
county_vax_stats = latest_covid_data.groupby('county').agg({
    'percent_of_population_fully_vaccinated': 'mean',
    'percent_of_population_partially_vaccinated': 'mean',
    'percent_of_population_with_1_plus_dose': 'mean',
    'tot_population': 'sum',
    'persons_fully_vaccinated': 'sum',
    'persons_partially_vaccinated': 'sum',
    'vaccine_equity_metric_quartile': 'mean'
}).reset_index()

print("County-level vaccination statistics:")
print(county_vax_stats.head())


Aggregating vaccination data to county level...


NameError: name 'latest_covid_data' is not defined

In [ ]:
# 5. Join vaccination data with county boundaries and income data
print("\nJoining datasets...")

# First, let's check what columns we actually have in our datasets
print("County vaccination stats columns:", county_vax_stats.columns.tolist())
print("Counties GeoDataFrame columns:", counties_gdf.columns.tolist())
print("Income data columns:", income_df.columns.tolist())

# Clean county names for matching
county_vax_stats['county'] = county_vax_stats['county'].str.title()

# For counties_gdf, we need to find the appropriate county name column
# It might be 'name', 'NAME', 'county', 'COUNTY', etc.
county_name_cols = [col for col in counties_gdf.columns if 'name' in col.lower() or 'county' in col.lower()]
print("Potential county name columns:", county_name_cols)

if 'NAME' in counties_gdf.columns:
    counties_gdf['NAME'] = counties_gdf['NAME'].str.title()
elif len(county_name_cols) > 0:
    # Use the first appropriate column found
    county_col = county_name_cols[0]
    print(f"Using '{county_col}' as county name column")
    # Rename it to 'NAME' for consistency
    counties_gdf = counties_gdf.rename(columns={county_col: 'NAME'})
    counties_gdf['NAME'] = counties_gdf['NAME'].str.title()
else:
    # If no name column found, but we have OBJECTID, create a name column
    if 'OBJECTID' in counties_gdf.columns:
        print("Creating county names from OBJECTID")
        # Map OBJECTID to county names if possible
        objectid_to_name = {
            1: 'Alameda County',
            2: 'Alpine County',
            3: 'Amador County',
            # Add more mappings as needed
        }
        # Create NAME column or use a default naming scheme
        counties_gdf['NAME'] = counties_gdf['OBJECTID'].map(objectid_to_name).fillna(
            'County ' + counties_gdf['OBJECTID'].astype(str)
        )
    else:
        print("Warning: No county name column found. Creating a dummy NAME column.")
        counties_gdf['NAME'] = [f"County {i+1}" for i in range(len(counties_gdf))]

# Clean income data county names
income_df['County'] = income_df['County'].str.title()

# Add "County" to county_vax_stats county names if not present
county_vax_stats['county_name'] = county_vax_stats['county'].apply(
    lambda x: x if 'County' in x else f"{x} County"
)

# Print sample values to verify matching
print("\nSample county names for matching:")
print("Vaccination data:", county_vax_stats['county_name'].unique()[:5])
print("GeoDataFrame:", counties_gdf['NAME'].unique()[:5])
print("Income data:", income_df['County'].unique()[:5])

# Merge vaccination data with county boundaries
counties_with_vax = counties_gdf.merge(
    county_vax_stats,
    left_on='NAME',
    right_on='county_name',
    how='left'
)

# Extract just a few key income metrics for simplicity
# Check if the columns exist first
available_income_cols = [col for col in ['AMI', 'LI_1', 'VLI_1', 'ELI_1'] if col in income_df.columns]
if not available_income_cols:
    print("Warning: Expected income columns not found. Using first 3 numeric columns instead.")
    income_cols = income_df.select_dtypes(include=['number']).columns[:3].tolist()
    income_metrics = income_df[['County'] + income_cols].copy()
else:
    income_metrics = income_df[['County'] + available_income_cols].copy()

# Create consistent county name format
income_metrics['county_name'] = income_metrics['County'].apply(
    lambda x: x if 'County' in x else f"{x} County"
)

counties_with_all_data = counties_with_vax.merge(
    income_metrics,
    left_on='NAME',
    right_on='county_name',
    how='left'
)

print("Combined dataset shape:", counties_with_all_data.shape)
print("Combined dataset columns:", counties_with_all_data.columns.tolist())
print(counties_with_all_data.head())


Joining datasets...


NameError: name 'county_vax_stats' is not defined

In [ ]:
# 6. Exploratory Data Analysis
print("\nPerforming exploratory analysis...")

# First, let's inspect what data we have in our datasets
print("COVID data sample:")
print(county_vax_stats.head())
print("\nCounties GeoDataFrame sample:")
print(counties_gdf.head())
print("\nIncome data sample:")
print(income_df.head())

# Since the join didn't work properly, let's try a different approach
# First, check for common county identifiers across datasets
print("\nChecking column values for matching:")
if 'county' in county_vax_stats.columns:
    print("Unique counties in vaccination data:", county_vax_stats['county'].unique()[:5], "...")
if 'NAME' in counties_gdf.columns:
    print("Unique names in spatial data:", counties_gdf['NAME'].unique()[:5], "...")
if 'County' in income_df.columns:
    print("Unique counties in income data:", income_df['County'].unique()[:5], "...")

# Manual approach to create a better joined dataset
# This will help us see why the joins are failing
data_combined = pd.DataFrame()

# Check if we have county-level aggregated vaccination data
if 'county' in county_vax_stats.columns:
    # Start with the vaccination statistics
    data_combined = county_vax_stats.copy()

    # Print column names to help debug
    print("\nColumns in county vaccination stats:")
    print(county_vax_stats.columns.tolist())

    # Ensure county names are consistent
    data_combined['county_clean'] = data_combined['county'].str.strip().str.title()
    # Remove "County" suffix if it exists for consistent matching
    data_combined['county_match'] = data_combined['county_clean'].apply(
        lambda x: x.replace(' County', '') if isinstance(x, str) else x
    )

    # Try to add income data
    if 'County' in income_df.columns:
        # Clean income county names
        income_df['county_clean'] = income_df['County'].str.strip().str.title()
        income_df['county_match'] = income_df['county_clean'].apply(
            lambda x: x.replace(' County', '') if isinstance(x, str) else x
        )

        # Print for debugging
        print("\nUnique counties after cleaning:")
        print("In vaccination data:", data_combined['county_match'].unique()[:5], "...")
        print("In income data:", income_df['county_match'].unique()[:5], "...")

        # Merge income data on cleaned county names
        income_cols = ['county_match', 'AMI', 'LI_1', 'VLI_1', 'ELI_1']
        income_cols = [col for col in income_cols if col in income_df.columns]

        if len(income_cols) > 1:  # Need at least county_match and one metric
            data_combined = data_combined.merge(
                income_df[income_cols],
                on='county_match',
                how='left'
            )

    # Check if we have data after merging
    print("\nData after merging income information:")
    print(data_combined.head())
    print(f"Shape: {data_combined.shape}")

    # Now add spatial data if available
    if isinstance(counties_gdf, gpd.GeoDataFrame) and 'geometry' in counties_gdf.columns:
        # Create a clean county name for matching
        if 'NAME' in counties_gdf.columns:
            counties_gdf['county_match'] = counties_gdf['NAME'].str.strip().str.title().apply(
                lambda x: x.replace(' County', '') if isinstance(x, str) else x
            )
        elif 'name' in counties_gdf.columns:
            counties_gdf['county_match'] = counties_gdf['name'].str.strip().str.title().apply(
                lambda x: x.replace(' County', '') if isinstance(x, str) else x
            )

        # If we have county_match column, merge geometry
        if 'county_match' in counties_gdf.columns:
            print("\nUnique counties in spatial data after cleaning:",
                  counties_gdf['county_match'].unique()[:5], "...")

            # Create a GeoDataFrame with our combined data
            # First get a list of counties that appear in both datasets
            common_counties = set(data_combined['county_match']).intersection(set(counties_gdf['county_match']))
            print(f"\nFound {len(common_counties)} common counties for mapping")

            if common_counties:
                # Filter to matching counties
                counties_subset = counties_gdf[counties_gdf['county_match'].isin(common_counties)]
                data_subset = data_combined[data_combined['county_match'].isin(common_counties)]

                # Merge based on the common counties
                geo_combined = counties_subset.merge(
                    data_subset,
                    on='county_match',
                    how='inner'
                )

                # Check if we have a valid GeoDataFrame with data
                if isinstance(geo_combined, gpd.GeoDataFrame) and len(geo_combined) > 0:
                    print(f"Successfully created GeoDataFrame with {len(geo_combined)} counties")
                    counties_with_all_data = geo_combined
                else:
                    print("Failed to create valid GeoDataFrame")
                    counties_with_all_data = data_combined
            else:
                print("No common counties found, skipping spatial join")
                counties_with_all_data = data_combined
        else:
            print("No county matching column in spatial data, skipping spatial join")
            counties_with_all_data = data_combined
    else:
        print("No valid geometry data available, continuing with tabular analysis only")
        counties_with_all_data = data_combined
else:
    print("No county-level vaccination data available, cannot proceed with analysis")
    # Create an empty DataFrame to avoid errors
    counties_with_all_data = pd.DataFrame()

# Check if we have anything to analyze
if counties_with_all_data.empty:
    print("WARNING: No data available for analysis after merging")
else:
    # Check columns in our final dataset
    print("\nColumns in final analysis dataset:")
    print(counties_with_all_data.columns.tolist())

    # Basic statistics of vaccination rates (if column exists)
    if 'percent_of_population_fully_vaccinated' in counties_with_all_data.columns:
        vax_data = counties_with_all_data['percent_of_population_fully_vaccinated'].dropna()
        if not vax_data.empty:
            vax_stats = vax_data.describe()
            print("\nVaccination rate statistics:")
            print(vax_stats)
        else:
            print("\nNo valid vaccination rate data available")
    else:
        print("\nVaccination rate column not found in final dataset")

    # Only create plots if we have data
    if not counties_with_all_data.empty and 'percent_of_population_fully_vaccinated' in counties_with_all_data.columns:
        if counties_with_all_data['percent_of_population_fully_vaccinated'].notna().any():
            # Create a figure for multiple plots
            plt.figure(figsize=(18, 12))

            # Histogram of vaccination rates
            plt.subplot(2, 2, 1)
            sns.histplot(counties_with_all_data['percent_of_population_fully_vaccinated'].dropna(), kde=True)
            plt.title('Distribution of Full Vaccination Rates by County')
            plt.xlabel('Percent Fully Vaccinated')

            # Boxplot of vaccination rates by vaccine equity metric quartile
            plt.subplot(2, 2, 2)
            if 'vaccine_equity_metric_quartile' in counties_with_all_data.columns and counties_with_all_data['vaccine_equity_metric_quartile'].notna().any():
                # Convert to string category to avoid numeric ordering issues
                counties_with_all_data['vem_category'] = counties_with_all_data['vaccine_equity_metric_quartile'].astype(str)

                # Check we have at least one value in each category
                vem_cats = counties_with_all_data.groupby('vem_category')['percent_of_population_fully_vaccinated'].count()
                if (vem_cats > 0).all():
                    sns.boxplot(x='vem_category',
                                y='percent_of_population_fully_vaccinated',
                                data=counties_with_all_data)
                    plt.title('Vaccination Rates by Equity Metric Quartile')
                    plt.xlabel('Vaccine Equity Metric Quartile')
                    plt.ylabel('Percent Fully Vaccinated')
                else:
                    plt.text(0.5, 0.5, 'Insufficient data in categories',
                             horizontalalignment='center', verticalalignment='center')
                    plt.title('No Data for Boxplot')
            else:
                plt.text(0.5, 0.5, 'Equity metric data not available',
                         horizontalalignment='center', verticalalignment='center')
                plt.title('Missing Data')

            # Scatter plot of vaccination rate vs median income (AMI)
            plt.subplot(2, 2, 3)
            if 'AMI' in counties_with_all_data.columns and counties_with_all_data['AMI'].notna().any():
                # Filter to rows where both columns have values
                scatter_data = counties_with_all_data[
                    counties_with_all_data['percent_of_population_fully_vaccinated'].notna() &
                    counties_with_all_data['AMI'].notna()
                ]

                if len(scatter_data) > 1:  # Need at least two points for scatter
                    sns.scatterplot(x='AMI',
                                    y='percent_of_population_fully_vaccinated',
                                    data=scatter_data)
                    plt.title('Vaccination Rate vs. Area Median Income')
                    plt.xlabel('Area Median Income (AMI)')
                    plt.ylabel('Percent Fully Vaccinated')
                else:
                    plt.text(0.5, 0.5, 'Insufficient paired data points',
                             horizontalalignment='center', verticalalignment='center')
                    plt.title('Insufficient Data')
            else:
                plt.text(0.5, 0.5, 'Income data not available',
                         horizontalalignment='center', verticalalignment='center')
                plt.title('Missing Data')

            # Box plots comparing urban vs rural counties based on population
            plt.subplot(2, 2, 4)
            if 'tot_population' in counties_with_all_data.columns and counties_with_all_data['tot_population'].notna().any():
                # Check if we have enough data for quantiles
                pop_data = counties_with_all_data['tot_population'].dropna()
                if len(pop_data) >= 3:
                    # Create a copy to avoid SettingWithCopyWarning
                    plot_data = counties_with_all_data.copy()

                    # Create population categories
                    try:
                        plot_data['population_category'] = pd.qcut(
                            plot_data['tot_population'].rank(method='first'),
                            q=3,
                            labels=['Low Population', 'Medium Population', 'High Population']
                        )

                        # Check we have data in each category
                        pop_cats = plot_data.groupby('population_category')['percent_of_population_fully_vaccinated'].count()
                        if (pop_cats > 0).all():
                            sns.boxplot(x='population_category',
                                        y='percent_of_population_fully_vaccinated',
                                        data=plot_data)
                            plt.title('Vaccination Rates by Population Size')
                            plt.xlabel('Population Category')
                            plt.ylabel('Percent Fully Vaccinated')
                        else:
                            plt.text(0.5, 0.5, 'Insufficient data in categories',
                                     horizontalalignment='center', verticalalignment='center')
                            plt.title('Category Data Insufficient')
                    except Exception as e:
                        plt.text(0.5, 0.5, f'Error creating population categories: {str(e)}',
                                 horizontalalignment='center', verticalalignment='center')
                        plt.title('Error in Categorization')
                else:
                    plt.text(0.5, 0.5, 'Insufficient population data for categories',
                             horizontalalignment='center', verticalalignment='center')
                    plt.title('Insufficient Data')
            else:
                plt.text(0.5, 0.5, 'Population data not available',
                         horizontalalignment='center', verticalalignment='center')
                plt.title('Missing Data')

            plt.tight_layout()
            plt.savefig('vaccination_eda_plots.png')
            plt.show()
        else:
            print("No valid vaccination data available for plotting")
    else:
        print("No data available for plotting")


Performing exploratory analysis...
COVID data sample:


NameError: name 'county_vax_stats' is not defined

In [ ]:
# 7. Correlation Analysis
print("\nPerforming correlation analysis...")

# First, let's check what columns we have in our dataset
print("Available columns:", counties_with_all_data.columns.tolist())

# Check which numeric columns are available
available_numeric_cols = []
for col in ['percent_of_population_fully_vaccinated',
            'percent_of_population_partially_vaccinated',
            'vaccine_equity_metric_quartile',
            'AMI', 'LI_1', 'VLI_1', 'ELI_1']:
    if col in counties_with_all_data.columns and counties_with_all_data[col].notna().any():
        available_numeric_cols.append(col)

if len(available_numeric_cols) >= 2:
    print(f"Using these columns for correlation analysis: {available_numeric_cols}")

    # Select numeric columns for correlation analysis
    correlation_data = counties_with_all_data[available_numeric_cols].dropna()

    if not correlation_data.empty and len(correlation_data) >= 3:  # Need at least 3 data points
        # Calculate correlation matrix
        correlation_matrix = correlation_data.corr()
        print("Correlation matrix:")
        print(correlation_matrix)

        # Visualize correlation matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
        plt.title('Correlation Matrix of Vaccination and Socioeconomic Variables')
        plt.tight_layout()
        plt.savefig('correlation_matrix.png')
        plt.show()

        # Calculate correlations if we have vaccination and income data
        if all(col in correlation_data.columns for col in ['percent_of_population_fully_vaccinated', 'AMI']):
            try:
                # Pearson correlation between vaccination rate and AMI
                pearson_corr, p_value = stats.pearsonr(
                    correlation_data['percent_of_population_fully_vaccinated'],
                    correlation_data['AMI']
                )
                print(f"Pearson correlation between vaccination rate and AMI: {pearson_corr:.3f} (p-value: {p_value:.4f})")

                # Spearman correlation (non-parametric, rank-based)
                spearman_corr, p_value = stats.spearmanr(
                    correlation_data['percent_of_population_fully_vaccinated'],
                    correlation_data['AMI']
                )
                print(f"Spearman correlation between vaccination rate and AMI: {spearman_corr:.3f} (p-value: {p_value:.4f})")
            except Exception as e:
                print(f"Error calculating correlations: {str(e)}")
        else:
            print("Cannot calculate correlations: missing either vaccination rate or AMI data")
    else:
        print("Warning: Correlation data is empty or insufficient after dropping missing values")
        print(f"Found {len(correlation_data)} complete rows for correlation analysis")
else:
    print(f"Not enough numeric columns available for correlation analysis. Available: {available_numeric_cols}")



Performing correlation analysis...


NameError: name 'counties_with_all_data' is not defined

In [ ]:
# 8. Regression Analysis
print("\nPerforming regression analysis...")

# Check if we have the necessary columns for regression
required_cols = ['AMI', 'vaccine_equity_metric_quartile', 'percent_of_population_fully_vaccinated']
missing_reg_cols = [col for col in required_cols if col not in counties_with_all_data.columns]

if missing_reg_cols:
    print(f"Missing columns for regression: {missing_reg_cols}")

    # Check if we can do simple regression with just vaccination and AMI
    if all(col in counties_with_all_data.columns for col in ['AMI', 'percent_of_population_fully_vaccinated']):
        print("Attempting simple regression with just AMI and vaccination rate")
        regression_data = counties_with_all_data[['AMI', 'percent_of_population_fully_vaccinated']].dropna()

        if len(regression_data) >= 5:  # Need at least a few data points for regression
            # Simple regression model
            X = regression_data[['AMI']]
            y = regression_data['percent_of_population_fully_vaccinated']

            # Add constant for intercept
            X_with_const = sm.add_constant(X)

            # Fit model
            try:
                model = sm.OLS(y, X_with_const).fit()
                print(model.summary())

                # Create regression scatter plot with line
                plt.figure(figsize=(10, 6))
                sns.regplot(x='AMI', y='percent_of_population_fully_vaccinated', data=regression_data)
                plt.title('Regression: Vaccination Rate vs Area Median Income')
                plt.xlabel('Area Median Income (AMI)')
                plt.ylabel('Percent Fully Vaccinated')
                plt.grid(True, alpha=0.3)
                plt.savefig('regression_plot.png')
                plt.show()
            except Exception as e:
                print(f"Error in simple regression: {str(e)}")
        else:
            print(f"Insufficient data for simple regression: only {len(regression_data)} complete rows")
    else:
        print("Cannot perform any regression: missing both AMI and vaccination rate data")
else:
    # We have all required columns, try multiple regression
    regression_data = counties_with_all_data[required_cols].dropna()

    if len(regression_data) >= 5:  # Need at least a few data points for regression
        # Multiple regression model
        try:
            X = regression_data[['AMI', 'vaccine_equity_metric_quartile']]
            y = regression_data['percent_of_population_fully_vaccinated']

            # Add constant for intercept
            X_with_const = sm.add_constant(X)

            # Fit model
            model = sm.OLS(y, X_with_const).fit()
            print(model.summary())

            # Create regression scatter plot with line
            plt.figure(figsize=(10, 6))
            sns.regplot(x='AMI', y='percent_of_population_fully_vaccinated', data=regression_data)
            plt.title('Regression: Vaccination Rate vs Area Median Income')
            plt.xlabel('Area Median Income (AMI)')
            plt.ylabel('Percent Fully Vaccinated')
            plt.grid(True, alpha=0.3)
            plt.savefig('regression_plot.png')
            plt.show()
        except Exception as e:
            print(f"Error in multiple regression: {str(e)}")
    else:
        print(f"Insufficient data for multiple regression: only {len(regression_data)} complete rows")



Performing regression analysis...


NameError: name 'counties_with_all_data' is not defined

In [ ]:
# 9. Spatial Analysis
print("\nPerforming spatial analysis...")

# Check if we have spatial data
if isinstance(counties_with_all_data, gpd.GeoDataFrame) and 'geometry' in counties_with_all_data.columns:
    print("GeoDataFrame with geometry available")

    # Make sure we have a valid GeoDataFrame with geometry
    spatial_data = counties_with_all_data.copy()

    # Check for missing geometries
    if spatial_data.geometry.isna().any():
        print(f"Warning: {spatial_data.geometry.isna().sum()} counties have missing geometry data")
        # Filter out rows with missing geometry
        spatial_data = spatial_data.dropna(subset=['geometry'])
        print(f"Filtered to {len(spatial_data)} counties with valid geometry")

    # Check if we have vaccination data
    if 'percent_of_population_fully_vaccinated' in spatial_data.columns:
        # Check how many counties have vaccination data
        vax_count = spatial_data['percent_of_population_fully_vaccinated'].notna().sum()
        print(f"Found {vax_count} counties with vaccination data")
x
        if vax_count >= 4:  # Need at least 4 counties for spatial analysis
            try:
                # Filter data to remove NaN vaccination rates
                spatial_data = spatial_data.dropna(subset=['percent_of_population_fully_vaccinated'])
                print(f"After removing NaNs, {len(spatial_data)} counties remain for spatial analysis")

                if len(spatial_data) >= 4:
                    # Create spatial weights matrix for counties
                    # Queen contiguity: counties that share any boundary point
                    W = Queen.from_dataframe(spatial_data)

                    # Calculate Moran's I for vaccination rates (testing for spatial autocorrelation)
                    moran = Moran(spatial_data['percent_of_population_fully_vaccinated'], W)
                    print(f"Moran's I: {moran.I:.3f}")
                    print(f"p-value: {moran.p_sim:.4f}")

                    # Plot Moran's I scatter plot
                    plt.figure(figsize=(10, 8))
                    plot_moran(moran, zstandard=True, figsize=(10, 8))
                    plt.savefig('morans_i_plot.png')
                    plt.show()

                    # Create choropleth map of vaccination rates
                    fig, ax = plt.subplots(figsize=(15, 10))
                    spatial_data.plot(
                        column='percent_of_population_fully_vaccinated',
                        cmap='YlGnBu',
                        linewidth=0.8,
                        ax=ax,
                        edgecolor='0.8',
                        legend=True,
                        legend_kwds={'label': "Percent Fully Vaccinated"}
                    )
                    ax.set_title('COVID-19 Vaccination Rates by County in California', fontsize=15)
                    ax.set_axis_off()
                    plt.savefig('vaccination_choropleth.png', dpi=300, bbox_inches='tight')
                    plt.show()

                    # 10. Bivariate Analysis - Create bivariate choropleth showing both vaccination and income
                    if 'AMI' in spatial_data.columns and spatial_data['AMI'].notna().any():
                        # Check if we have enough data for quantiles
                        ami_count = spatial_data['AMI'].notna().sum()
                        print(f"Found {ami_count} counties with income data")

                        # Need at least 3 counties in each quantile (so 9 total)
                        if len(spatial_data.dropna(subset=['percent_of_population_fully_vaccinated', 'AMI'])) >= 9:
                            try:
                                # Create a copy for bivariate analysis to avoid SettingWithCopyWarning
                                bivar_data = spatial_data.dropna(subset=['percent_of_population_fully_vaccinated', 'AMI']).copy()

                                # Create quantiles for both variables
                                bivar_data['vax_quantile'] = pd.qcut(
                                    bivar_data['percent_of_population_fully_vaccinated'].rank(method='first'),
                                    3,
                                    labels=['Low', 'Medium', 'High']
                                )

                                bivar_data['income_quantile'] = pd.qcut(
                                    bivar_data['AMI'].rank(method='first'),
                                    3,
                                    labels=['Low', 'Medium', 'High']
                                )

                                # Create a bivariate category
                                bivar_data['bivariate_category'] = bivar_data['vax_quantile'].astype(str) + '-' + bivar_data['income_quantile'].astype(str)

                                # Map categories to color values (3x3 grid)
                                bivariate_colors = {
                                    'Low-Low': '#e8e8e8',      # Light gray
                                    'Low-Medium': '#ace4e4',   # Light blue
                                    'Low-High': '#5ac8c8',     # Medium blue
                                    'Medium-Low': '#dfb0d6',   # Light purple
                                    'Medium-Medium': '#a5add3', # Medium purple
                                    'Medium-High': '#5698b9',   # Blue-purple
                                    'High-Low': '#be64ac',     # Pink
                                    'High-Medium': '#8c62aa',  # Purple
                                    'High-High': '#3b4994'     # Dark blue
                                }

                                bivar_data['bivariate_color'] = bivar_data['bivariate_category'].map(bivariate_colors)

                                # Print category counts to verify distribution
                                print("Bivariate category counts:")
                                print(bivar_data['bivariate_category'].value_counts())

                                # Plot bivariate choropleth
                                fig, ax = plt.subplots(figsize=(15, 10))
                                bivar_data.plot(
                                    color=bivar_data['bivariate_color'],
                                    linewidth=0.8,
                                    ax=ax,
                                    edgecolor='0.8'
                                )

                                # Create a custom legend
                                from matplotlib.patches import Patch
                                legend_elements = [
                                    Patch(facecolor=color, label=cat)
                                    for cat, color in bivariate_colors.items()
                                    if cat in bivar_data['bivariate_category'].values
                                ]
                                ax.legend(handles=legend_elements,
                                          title="Vaccination-Income Categories\n(Vaccination-Income)",
                                          loc='lower right', frameon=True)

                                ax.set_title('Bivariate Map of COVID-19 Vaccination Rates and Income by County', fontsize=15)
                                ax.set_axis_off()
                                plt.savefig('bivariate_choropleth.png', dpi=300, bbox_inches='tight')
                                plt.show()
                            except Exception as e:
                                print(f"Error in bivariate analysis: {str(e)}")
                        else:
                            print("Insufficient data for bivariate analysis after removing missing values")
                    else:
                        print("Cannot perform bivariate analysis: missing AMI column or all values are NaN")
                else:
                    print("After filtering, insufficient counties remain for spatial analysis")
            except Exception as e:
                print(f"Error in spatial analysis: {str(e)}")
                import traceback
                traceback.print_exc()
        else:
            print("Insufficient counties with vaccination data for spatial analysis")
    else:
        print("Cannot perform spatial analysis: missing vaccination rate column")
else:
    print("Cannot perform spatial analysis: missing geometry column or not a GeoDataFrame")
    print(f"Type of counties_with_all_data: {type(counties_with_all_data)}")
    if 'geometry' not in counties_with_all_data.columns:
        print("No geometry column in dataset")

# Save processed datasets if we have data
try:
    print("\nSaving processed datasets...")
    # Check if we have a GeoDataFrame
    if isinstance(counties_with_all_data, gpd.GeoDataFrame) and 'geometry' in counties_with_all_data.columns:
        print("Saving as GeoJSON...")
        counties_with_all_data.to_file('california_counties_covid_income_analysis.geojson', driver='GeoJSON')

    # Save as CSV (without geometry column if it exists)
    if 'geometry' in counties_with_all_data.columns:
        print("Saving CSV without geometry column...")
        counties_with_all_data.drop('geometry', axis=1).to_csv('california_counties_covid_income_analysis.csv', index=False)
    else:
        print("Saving as CSV...")
        counties_with_all_data.to_csv('california_counties_covid_income_analysis.csv', index=False)

    print("Output files have been saved.")
except Exception as e:
    print(f"Error saving data: {str(e)}")

# 13. Generate summary statistics for the paper
print("\nGenerating summary statistics for the research paper...")

# Initialize empty dictionary for stats
summary_stats = {'total_counties': len(counties_with_all_data)}

# Add vaccination statistics if available
if 'percent_of_population_fully_vaccinated' in counties_with_all_data.columns:
    vax_data = counties_with_all_data['percent_of_population_fully_vaccinated'].dropna()
    if not vax_data.empty:
        summary_stats.update({
            'mean_vax_rate': vax_data.mean(),
            'median_vax_rate': vax_data.median(),
            'min_vax_rate': vax_data.min(),
            'max_vax_rate': vax_data.max(),
            'std_vax_rate': vax_data.std(),
            'counties_with_vax_data': len(vax_data)
        })
    else:
        summary_stats['counties_with_vax_data'] = 0

# Add correlation stats if they were calculated
if 'pearson_corr' in locals() and 'p_value' in locals():
    summary_stats.update({
        'pearson_corr_vax_income': pearson_corr,
        'pearson_pvalue': p_value
    })

# Add Moran's I stats if they were calculated
if 'moran' in locals():
    summary_stats.update({
        'morans_i': moran.I,
        'morans_i_pvalue': moran.p_sim
    })

print("Summary Statistics for Research Paper:")
for key, value in summary_stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

# 14. Additional visualizations if we have enough data
print("\nCreating additional visualizations for the paper...")

# Check which variables are available
available_vars = []
for col in ['percent_of_population_fully_vaccinated', 'AMI',
            'vaccine_equity_metric_quartile', 'tot_population']:
    if col in counties_with_all_data.columns and counties_with_all_data[col].notna().any():
        available_vars.append(col)

if len(available_vars) >= 2:
    print(f"Can create scatter matrix with these variables: {available_vars}")

    # Create a copy of the data with just these variables
    scatter_data = counties_with_all_data[available_vars].copy().dropna()

    if len(scatter_data) >= 3:  # Need at least a few points for a meaningful scatter matrix
        # Create more user-friendly column names
        col_mapping = {
            'percent_of_population_fully_vaccinated': 'Vaccination Rate',
            'AMI': 'Median Income',
            'vaccine_equity_metric_quartile': 'Equity Metric',
            'tot_population': 'Population'
        }

        rename_cols = {col: col_mapping.get(col, col) for col in scatter_data.columns if col in col_mapping}
        scatter_data = scatter_data.rename(columns=rename_cols)

        plt.figure(figsize=(12, 10))
        scatter_matrix = pd.plotting.scatter_matrix(
            scatter_data,
            alpha=0.8,
            figsize=(12, 10),
            diagonal='kde'
        )
        plt.suptitle('Scatter Plot Matrix of Key Variables', fontsize=16, y=0.95)
        plt.tight_layout()
        plt.savefig('scatter_matrix.png', dpi=300)
        plt.show()
    else:
        print(f"Insufficient data for scatter plot matrix: only {len(scatter_data)} complete rows")
else:
    print(f"Not enough variables available for scatter matrix. Available: {available_vars}")


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 147)

In [ ]:
W = Queen.from_dataframe(counties_with_all_data)

y = counties_with_all_data['percent_of_population_fully_vaccinated']
lisa = Moran_Local(y, W)

fig, ax = plt.subplots(1, figsize=(12, 8))
lisa_cluster(lisa, counties_with_all_data, p=0.05, ax=ax)
plt.title("Local Moran's I Cluster Map (Vaccination Rates)")
plt.show()
sample_county = 'Los Angeles County'
county_data = time_series_data[time_series_data['county'] == sample_county]

plt.figure(figsize=(10, 6))
plt.plot(county_data['as_of_date'], county_data['percent_of_population_fully_vaccinated'], marker='o', linestyle='-')
plt.title(f'Vaccination Progress in {sample_county}')
plt.xlabel('Date')
plt.ylabel('Percent Fully Vaccinated')
plt.grid(True, alpha=0.5)
plt.show()

slopes = []
counties = time_series_data['county'].unique()

for county in counties:
    county_data = time_series_data[time_series_data['county'] == county]
    if len(county_data) >= 2:
        X = np.arange(len(county_data)).reshape(-1, 1)
        y = county_data['percent_of_population_fully_vaccinated'].values
        model = LinearRegression().fit(X, y)
        slopes.append((county, model.coef_[0]))

sorted_slopes = sorted(slopes, key=lambda x: x[1], reverse=True)
print("Top counties with fastest vaccination uptake:", sorted_slopes[:5])
print("Top counties with slowest vaccination uptake:", sorted_slopes[-5:])

NameError: name 'counties_with_all_data' is not defined

In [ ]:
W = Queen.from_dataframe(counties_with_all_data)

y = counties_with_all_data['percent_of_population_fully_vaccinated']
lisa = Moran_Local(y, W)

fig, ax = plt.subplots(1, figsize=(12, 8))
lisa_cluster(lisa, counties_with_all_data, p=0.05, ax=ax)
plt.title("Local Moran's I Cluster Map (Vaccination Rates)")
plt.show()

counties_with_all_data['vax_quantile'] = pd.qcut(
    counties_with_all_data['percent_of_population_fully_vaccinated'].rank(method='first'), 3, labels=['Low', 'Medium', 'High']
)
counties_with_all_data['income_quantile'] = pd.qcut(
    counties_with_all_data['AMI'].rank(method='first'), 3, labels=['Low', 'Medium', 'High']
)

counties_with_all_data['bivariate_category'] = counties_with_all_data['vax_quantile'].astype(str) + '-' + counties_with_all_data['income_quantile'].astype(str)
counties_with_all_data['bivariate_color'] = counties_with_all_data['bivariate_category'].map(bivariate_colors)

fig, ax = plt.subplots(figsize=(15, 10))
counties_with_all_data.plot(
    color=counties_with_all_data['bivariate_color'], linewidth=0.8, ax=ax, edgecolor='0.8'
)

legend_elements = [
    Patch(facecolor=color, label=cat)
    for cat, color in bivariate_colors.items() if cat in counties_with_all_data['bivariate_category'].values
]
ax.legend(handles=legend_elements, title="Vaccination-Income Categories\n(Vaccination-Income)", loc='lower right')
ax.set_title('Bivariate Map of COVID-19 Vaccination Rates and Income by County', fontsize=15)
ax.set_axis_off()
plt.show()

NameError: name 'counties_with_all_data' is not defined